In [1]:
import heapq

class GreedyRobotCargoGA:
    def __init__(self, grid_size, robot_positions, cargo_positions, cargo_destinations, time_steps):
            self.grid_size = grid_size
            self.initial_robot_positions = robot_positions
            self.initial_cargo_positions = cargo_positions
            self.cargo_destinations = cargo_destinations
            self.time_steps = time_steps
            self.blocked_positions = set()  # 一般的阻擋位置
            self.locked_positions = {}   # 貨物鎖死後的阻擋位置
            self.robot_paths = []
            self.available_cargoes = set(range(len(cargo_positions)))
            self.robot_status = [0] * len(robot_positions)  # 跟踪每個機器人的任務完成時間
            self.robot_tasks = {}  # 儲存每個機器人正在處理的貨物
            self.cargo_positions = cargo_positions[:]  # 複製以允許修改

    def block_path(self, path, cargo_id=None):
        for i in range(len(path) - 1):
            x1, y1, t1 = path[i]
            x2, y2, t2 = path[i + 1]


            self.blocked_positions.add((x1, y1, t1))
            self.blocked_positions.add((x2, y2, t2))
            
            # 封鎖雙向路徑，無論有無貨物
            self.blocked_positions.add(((x1, y1), (x2, y2), t1))
            self.blocked_positions.add(((x2, y2), (x1, y1), t1))
            if (x1, y1, t1) == (x2, y2, t2):
                # 上 (y1-1)
                self.blocked_positions.add(((x1, y1), (x1, y1-1), t1))
                self.blocked_positions.add(((x1, y1-1), (x1, y1), t1))

                # 下 (y1+1)
                self.blocked_positions.add(((x1, y1), (x1, y1+1), t1))
                self.blocked_positions.add(((x1, y1+1), (x1, y1), t1))

                # 左 (x1-1)
                self.blocked_positions.add(((x1, y1), (x1-1, y1), t1))
                self.blocked_positions.add(((x1-1, y1), (x1, y1), t1))

                # 右 (x1+1)
                self.blocked_positions.add(((x1, y1), (x1+1, y1), t1))
                self.blocked_positions.add(((x1+1, y1), (x1, y1), t1))


    def is_valid(self, x, y, t, prev_x=None, prev_y=None):
        # 確認座標是否在網格內
        if not (0 <= x < self.grid_size and 0 <= y < self.grid_size):
            return False

        # 檢查兩點之間的路徑是否被封鎖

        if prev_x is not None and prev_y is not None:
            if (((prev_x, prev_y), (x, y), t - 1) in self.blocked_positions or 
                ((x, y), (prev_x, prev_y), t - 1) in self.blocked_positions):
                return False
        if prev_x is not None and prev_y is not None:
            if (((prev_x, prev_y), (x, y), t) in self.blocked_positions or 
                ((x, y), (prev_x, prev_y), t) in self.blocked_positions):
                return False
        if prev_x is not None and prev_y is not None:
            if (((prev_x, prev_y), (x, y), t+1) in self.blocked_positions or 
                ((x, y), (prev_x, prev_y), t+1) in self.blocked_positions):
                return False


        # 檢查該點是否在時間t時被阻擋
        if (x, y, t) in self.blocked_positions:
            return False

        # 檢查該位置是否為已鎖定的貨物目的地
        if (x, y,t) in self.locked_positions:
            return False
        
        for robot_idx, cargo_idx in self.robot_tasks.items():
            cargo_pos = self.cargo_positions[cargo_idx]
            # 檢查該貨物是否在機器人運輸路徑上
            if cargo_pos == (x, y) and self.robot_status[robot_idx] > t:
                return False  # 如果有貨物在這裡且它正在被搬運，則這個位置是障礙物


        # 確認該時間點是否有其他機器人在此位置
        for robot_idx, path in self.robot_paths:
            for (rx, ry, rt) in path:
                if rt == t and (rx, ry) == (x, y):
                    return False  # 該位置被其他機器人佔據

        return True
    def neighbors(self, x, y, t):
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if self.is_valid(nx, ny, t + 1, x, y):
                yield nx, ny, t + 1

    def shortest_path(self, start, goal, start_time=0):
        queue = [(0, start[0], start[1], start_time, [])]  # (cost, x, y, time, path)
        visited = set()
        while queue:
            cost, x, y, t, path = heapq.heappop(queue)
            if (x, y, t) in visited:
                continue
            visited.add((x, y, t))
            path = path + [(x, y, t)]
            if (x, y) == goal:
                return cost, path
            for nx, ny, nt in self.neighbors(x, y, t):
                heapq.heappush(queue, (cost + 1, nx, ny, nt, path))
        return float('inf'), []
    
    def is_locked(self, x, y, current_time):
        """檢查位置是否已經鎖定，僅當當前時間超過鎖定時間時才真正鎖定"""
        if (x, y) in self.locked_positions:
            lock_time = self.locked_positions[(x, y)]
            if current_time >= lock_time:

                return True
            else:

                return False
        return False  # 如果位置不在鎖定列表中，則視為未鎖定

    def lock_position(self, x, y, lock_time):
        """在指定時間後鎖定已運送完成的貨物位置，防止其他機器人干擾"""
        self.locked_positions[(x, y)] = lock_time  # 位置和鎖定時間


    def find_nearest_empty_position(self, start_pos):
        min_dist = float('inf')
        nearest_pos = None
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                pos = (x, y)
                if pos not in self.cargo_positions and (x, y) not in self.blocked_positions:
                    dist, _ = self.shortest_path(start_pos, pos)
                    if dist < min_dist:
                        min_dist = dist
                        nearest_pos = pos
        return nearest_pos

    def move_blocking_cargo(self, robot_idx, start_pos, cargo_id):
        # 獲取目標貨物的目的地
        target_cargo_pos = self.cargo_positions[cargo_id]
        target_cargo_dest = self.cargo_destinations[cargo_id]

        # 獲取到達目的地的路徑
        current_time = self.robot_status[robot_idx]
        
        path_to_target = self.shortest_path(start_pos, target_cargo_pos,current_time)[1]
        total_path = [] 
        path_to_dest = self.shortest_path(target_cargo_pos, target_cargo_dest,current_time)[1]
        total_path.extend([(x, y, t) for (x, y, t) in path_to_target])
        current_time += len(path_to_target)
        block = False        # 檢查路徑上的障礙物
        for x, y, t in path_to_dest:
            
            if (x, y) in self.cargo_positions and (x, y) != self.cargo_positions[cargo_id]:

                block = True
                blocking_cargo_idx = self.cargo_positions.index((x, y))  # 獲取阻擋的貨物索引
                path_to_block = self.shortest_path(target_cargo_pos, (x, y),current_time)[1]

                total_path.extend([(x, y, t) for (x, y, t) in path_to_block])
                current_time += len(path_to_block)
                # 尋找空位
                empty_pos = self.find_nearest_empty_position((x, y))
                if empty_pos:
                    # 計算從阻擋貨物位置到空位的路徑
                    path_to_empty = self.shortest_path((x, y), empty_pos,current_time)[1]
                    path_to_empty = path_to_empty[1:]
                    #self.block_path(path_to_empty)
                    total_path.extend([(x, y, t-1) for (x, y, t) in path_to_empty])
                    current_time += len(path_to_empty)
                    

                    # 更新貨物位置
                    self.cargo_positions[blocking_cargo_idx] = empty_pos
                    self.update_cargo_positions()
                    
                    # 更新回到原來貨物位置的路徑
                    path_back_to_cargo = self.shortest_path(empty_pos,(x, y),current_time)[1]
                    path_back_to_cargo = path_back_to_cargo[1:]
                    #self.block_path(path_back_to_cargo)

                    total_path.extend([(x, y, t-1) for (x, y, t) in path_back_to_cargo])
                    current_time += len(path_back_to_cargo)
                    place = (x,y) 
        if block and empty_pos:
            path_back_to_cargo = self.shortest_path(place,target_cargo_pos,current_time)[1]
            path_back_to_cargo = path_back_to_cargo[1:]
            #self.block_path(path_back_to_cargo)

            
            total_path.extend([(x, y, t-1) for (x, y, t) in path_back_to_cargo])
            current_time += len(path_back_to_cargo) 
        # 在最終路徑上進行搬移貨物的操作
        path_to_dest = self.shortest_path(target_cargo_pos, target_cargo_dest,current_time)[1]
        path_to_dest = path_to_dest[1:]
        total_path.extend([(x, y, t-1) for (x, y, t) in path_to_dest])
        current_time += len(path_to_dest)
        # 更新機器人的狀態

        self.robot_status[robot_idx] += len(total_path)

        self.block_path(total_path)

        return total_path

    def allocate_tasks(self, assignments):
        time = 0  # 初始化時間
        task_assignment = []  # 紀錄任務分配和路徑
        available_robots = list(range(len(self.initial_robot_positions)))  # 可用的機器人索引列表
        self.available_cargoes = {cargo_idx for cargo_idx in self.available_cargoes 
                                      if self.cargo_destinations[cargo_idx] != (float('inf'), float('inf'))}

        while time < self.time_steps and assignments:
            # 使用副本迭代以避免修改問題
            for (robot_idx, cargo_idx) in assignments[:]:  
                # 檢查機器人是否可以執行任務且當前時間允許
                if time >= self.robot_status[robot_idx]:
                    cargo_pos = self.cargo_positions[cargo_idx]
                    cargo_dest = self.cargo_destinations[cargo_idx]

                    # 移動阻擋的貨物並計算路徑
                    blocking_path = self.move_blocking_cargo(robot_idx, self.initial_robot_positions[robot_idx], cargo_idx)

                    # 將機器人路徑與阻擋路徑合併
                    robot_path = blocking_path

                    # 如果機器人路徑穿過貨物位置或目標位置，跳過此任務
                    if self.path_passes_through(robot_path, robot_idx, cargo_pos, cargo_dest):
                        continue
                    # 更新貨物和機器人狀態
                    self.available_cargoes.remove(cargo_idx)  # 移除完成的貨物
                    assignments.remove((robot_idx, cargo_idx))  # 移除已分配的任務
                    available_robots.remove(robot_idx)  # 標記機器人為不可用直到任務完成
                    if robot_path:
                        robot_pos = robot_path[-1]  # 機器人的最終位置
                        self.initial_robot_positions[robot_idx] = cargo_dest
                        self.cargo_positions[cargo_idx] = robot_pos
                        self.lock_position(cargo_dest[0], cargo_dest[1], robot_path[-1][2])  # 鎖定目的地
                        self.robot_paths.append((f"R{robot_idx + 1}", robot_path))

                        # 紀錄任務分配和路徑
                        task_assignment.append((robot_idx, cargo_idx, robot_path))  # 將機器人與貨物和路徑對應
                       
                        break  # 任務分配完成，退出循環

            # 增加時間步
            time += 1

            # 將完成任務的機器人標記為可用
            for robot_idx in range(len(self.robot_status)):
                if time >= self.robot_status[robot_idx] and robot_idx not in available_robots:
                    available_robots.append(robot_idx)  # 標記機器人為可用

        return task_assignment  # 返回 (robot_idx, cargo_idx, path) 列表




    def path_passes_through(self, path, current_robot_idx,cargo_pos,cargo_dest):
        """檢查路徑是否經過其他機器人的目標位置"""
        # 將可用貨物集合轉換為列表
        available_cargos_list = list(self.available_cargoes)


        # 檢查當前路徑的所有位置
        path_positions = set((x, y) for x, y, t in path)

        # 檢查是否有其他機器人的路徑會經過當前機器人的目標位置
        for robot_name, robot_path in self.robot_paths:
            if robot_name != f"R{current_robot_idx + 1}":  # 忽略當前機器人的路徑
                for (x, y, t) in robot_path:
                    if (x, y) == cargo_dest and t >= self.robot_status[current_robot_idx]-1:  # 確保時間大於當前時間
                        current_time = self.robot_status[current_robot_idx]
                        return True  # 檢測到經過其他機器人的目標位置
         
        

        return False  # 沒有經過


    def update_cargo_positions(self):
        new_positions = list(self.cargo_positions)

        for robot_idx, cargo_idx in self.robot_tasks.items():
            robot_pos = self.initial_robot_positions[robot_idx]

            if cargo_idx in self.cargo_positions:
                if self.cargo_positions[cargo_idx] == robot_pos:
                    new_positions[cargo_idx] = robot_pos

        self.cargo_positions = new_positions
    import heapq
import time
import numpy as np
class GreedyRobotCargo:
    def __init__(self, grid_size, robot_positions, cargo_positions, cargo_destinations, time_steps):
            self.grid_size = grid_size
            self.initial_robot_positions = robot_positions
            self.initial_cargo_positions = cargo_positions
            self.cargo_destinations = cargo_destinations
            self.time_steps = time_steps
            self.blocked_positions = set()  # 一般的阻擋位置
            self.locked_positions = {}   # 貨物鎖死後的阻擋位置
            self.robot_paths = []
            self.available_cargoes = set(range(len(cargo_positions)))
            self.robot_status = [0] * len(robot_positions)  # 跟踪每個機器人的任務完成時間
            self.robot_tasks = {}  # 儲存每個機器人正在處理的貨物
            self.cargo_positions = cargo_positions[:]  # 複製以允許修改

    def block_path(self, path, cargo_id=None):
        for i in range(len(path) - 1):
            x1, y1, t1 = path[i]
            x2, y2, t2 = path[i + 1]


            self.blocked_positions.add((x1, y1, t1))
            self.blocked_positions.add((x2, y2, t2))
            
            # 封鎖雙向路徑，無論有無貨物
            self.blocked_positions.add(((x1, y1), (x2, y2), t1))
            self.blocked_positions.add(((x2, y2), (x1, y1), t1))
            if (x1, y1, t1) == (x2, y2, t2):
                # 上 (y1-1)
                self.blocked_positions.add(((x1, y1), (x1, y1-1), t1))
                self.blocked_positions.add(((x1, y1-1), (x1, y1), t1))

                # 下 (y1+1)
                self.blocked_positions.add(((x1, y1), (x1, y1+1), t1))
                self.blocked_positions.add(((x1, y1+1), (x1, y1), t1))

                # 左 (x1-1)
                self.blocked_positions.add(((x1, y1), (x1-1, y1), t1))
                self.blocked_positions.add(((x1-1, y1), (x1, y1), t1))

                # 右 (x1+1)
                self.blocked_positions.add(((x1, y1), (x1+1, y1), t1))
                self.blocked_positions.add(((x1+1, y1), (x1, y1), t1))


    def is_valid(self, x, y, t, prev_x=None, prev_y=None):
        # 確認座標是否在網格內
        if not (0 <= x < self.grid_size and 0 <= y < self.grid_size):
            return False

        # 檢查兩點之間的路徑是否被封鎖

        if prev_x is not None and prev_y is not None:
            if (((prev_x, prev_y), (x, y), t - 1) in self.blocked_positions or 
                ((x, y), (prev_x, prev_y), t - 1) in self.blocked_positions):
                return False
        if prev_x is not None and prev_y is not None:
            if (((prev_x, prev_y), (x, y), t) in self.blocked_positions or 
                ((x, y), (prev_x, prev_y), t) in self.blocked_positions):
                return False
        if prev_x is not None and prev_y is not None:
            if (((prev_x, prev_y), (x, y), t+1) in self.blocked_positions or 
                ((x, y), (prev_x, prev_y), t+1) in self.blocked_positions):
                return False


        # 檢查該點是否在時間t時被阻擋
        if (x, y, t) in self.blocked_positions:
            return False

        # 檢查該位置是否為已鎖定的貨物目的地
        if (x, y,t) in self.locked_positions:
            return False
        
        for robot_idx, cargo_idx in self.robot_tasks.items():
            cargo_pos = self.cargo_positions[cargo_idx]
            # 檢查該貨物是否在機器人運輸路徑上
            if cargo_pos == (x, y) and self.robot_status[robot_idx] > t:
                return False  # 如果有貨物在這裡且它正在被搬運，則這個位置是障礙物


        # 確認該時間點是否有其他機器人在此位置
        for robot_idx, path in self.robot_paths:
            for (rx, ry, rt) in path:
                if rt == t and (rx, ry) == (x, y):
                    return False  # 該位置被其他機器人佔據

        return True
    def neighbors(self, x, y, t):
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if self.is_valid(nx, ny, t + 1, x, y):
                yield nx, ny, t + 1

    def shortest_path(self, start, goal, start_time=0):
        queue = [(0, start[0], start[1], start_time, [])]  # (cost, x, y, time, path)
        visited = set()
        while queue:
            cost, x, y, t, path = heapq.heappop(queue)
            if (x, y, t) in visited:
                continue
            visited.add((x, y, t))
            path = path + [(x, y, t)]
            if (x, y) == goal:
                return cost, path
            for nx, ny, nt in self.neighbors(x, y, t):
                heapq.heappush(queue, (cost + 1, nx, ny, nt, path))
        return float('inf'), []
    
    def is_locked(self, x, y, current_time):
        """檢查位置是否已經鎖定，僅當當前時間超過鎖定時間時才真正鎖定"""
        if (x, y) in self.locked_positions:
            lock_time = self.locked_positions[(x, y)]
            if current_time >= lock_time:
                print(f"位置 ({x}, {y}) 已鎖定，當前時間: {current_time}")
                return True
            else:
                print(f"位置 ({x}, {y}) 尚未鎖定，鎖定時間為 {lock_time}，當前時間: {current_time}")
                return False
        return False  # 如果位置不在鎖定列表中，則視為未鎖定

    def lock_position(self, x, y, lock_time):
        """在指定時間後鎖定已運送完成的貨物位置，防止其他機器人干擾"""
        self.locked_positions[(x, y)] = lock_time  # 位置和鎖定時間
        print(f"位置 ({x}, {y}) 將在時間 {lock_time} 之後鎖定，無法再搬動此貨物。")

    def find_nearest_empty_position(self, start_pos):
        min_dist = float('inf')
        nearest_pos = None
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                pos = (x, y)
                if pos not in self.cargo_positions and (x, y) not in self.blocked_positions:
                    dist, _ = self.shortest_path(start_pos, pos)
                    if dist < min_dist:
                        min_dist = dist
                        nearest_pos = pos
        return nearest_pos

    def move_blocking_cargo(self, robot_idx, start_pos, cargo_id):
        # 獲取目標貨物的目的地
        target_cargo_pos = self.cargo_positions[cargo_id]
        target_cargo_dest = self.cargo_destinations[cargo_id]

        # 獲取到達目的地的路徑
        current_time = self.robot_status[robot_idx]
        
        path_to_target = self.shortest_path(start_pos, target_cargo_pos,current_time)[1]
        total_path = [] 
        path_to_dest = self.shortest_path(target_cargo_pos, target_cargo_dest,current_time)[1]
        total_path.extend([(x, y, t) for (x, y, t) in path_to_target])
        current_time += len(path_to_target)
        block = False        # 檢查路徑上的障礙物
        for x, y, t in path_to_dest:
            
            if (x, y) in self.cargo_positions and (x, y) != self.cargo_positions[cargo_id]:

                block = True
                blocking_cargo_idx = self.cargo_positions.index((x, y))  # 獲取阻擋的貨物索引
                path_to_block = self.shortest_path(target_cargo_pos, (x, y),current_time)[1]

                total_path.extend([(x, y, t) for (x, y, t) in path_to_block])
                current_time += len(path_to_block)
                # 尋找空位
                empty_pos = self.find_nearest_empty_position((x, y))
                if empty_pos:
                    # 計算從阻擋貨物位置到空位的路徑
                    path_to_empty = self.shortest_path((x, y), empty_pos,current_time)[1]
                    path_to_empty = path_to_empty[1:]
                    #self.block_path(path_to_empty)
                    total_path.extend([(x, y, t-1) for (x, y, t) in path_to_empty])
                    current_time += len(path_to_empty)
                    
                    print(f"  moving: {self.cargo_positions[blocking_cargo_idx]} to {empty_pos}")
                    print(total_path)
                    # 更新貨物位置
                    self.cargo_positions[blocking_cargo_idx] = empty_pos
                    self.update_cargo_positions()
                    
                    # 更新回到原來貨物位置的路徑
                    path_back_to_cargo = self.shortest_path(empty_pos,(x, y),current_time)[1]
                    path_back_to_cargo = path_back_to_cargo[1:]
                    #self.block_path(path_back_to_cargo)
                    print(total_path)
                    total_path.extend([(x, y, t-1) for (x, y, t) in path_back_to_cargo])
                    current_time += len(path_back_to_cargo)
                    place = (x,y) 
        if block and empty_pos:
            path_back_to_cargo = self.shortest_path(place,target_cargo_pos,current_time)[1]
            path_back_to_cargo = path_back_to_cargo[1:]
            #self.block_path(path_back_to_cargo)
            print(total_path)
            
            total_path.extend([(x, y, t-1) for (x, y, t) in path_back_to_cargo])
            current_time += len(path_back_to_cargo)
        else:
            print("無解")
        # 在最終路徑上進行搬移貨物的操作
        path_to_dest = self.shortest_path(target_cargo_pos, target_cargo_dest,current_time)[1]
        path_to_dest = path_to_dest[1:]
        total_path.extend([(x, y, t-1) for (x, y, t) in path_to_dest])
        current_time += len(path_to_dest)
        # 更新機器人的狀態

        self.robot_status[robot_idx] += len(total_path)

        self.block_path(total_path)
        print(f"  Total path: {total_path}")

        return total_path

    def allocate_tasks(self):
        start_times = time.time()

        times = 0
        while self.available_cargoes:
            all_robots_busy = True

            # 只考慮未完成的貨物
            self.available_cargoes = {cargo_idx for cargo_idx in self.available_cargoes 
                                      if self.cargo_destinations[cargo_idx] != (float('inf'), float('inf'))}

            for robot_idx in range(len(self.initial_robot_positions)):
                if times >= self.robot_status[robot_idx]:  # 檢查機器人是否可用
                    all_robots_busy = False
                    if self.available_cargoes:
                        min_dist = float('inf')
                        target_cargo = None
                        best_path_to_cargo = None

                        # 為機器人選擇最近的可用貨物
                        for cargo_idx in self.available_cargoes:
                            cargo_pos = self.cargo_positions[cargo_idx]
                            start_time = self.robot_status[robot_idx]  # 機器人的當前可用時間
                            dist, path_to_cargo = self.shortest_path(self.initial_robot_positions[robot_idx], cargo_pos, start_time)

                            # 打印檢查的信息
                            print(f"Robot {robot_idx + 1} checking cargo {cargo_idx + 1}:")
                            print(f"  Cargo position: {cargo_pos}")
                            print(f"  Path to cargo: {path_to_cargo}")
                            print(f"  Distance: {dist}")

                            if dist < min_dist:
                                min_dist = dist
                                target_cargo = cargo_idx
                                best_path_to_cargo = path_to_cargo

                        if target_cargo is not None:
                            cargo_dest = self.cargo_destinations[target_cargo]
                            cargo_pos = self.cargo_positions[target_cargo]
                            print(self.robot_paths)
                            blocking_path = self.move_blocking_cargo(robot_idx, self.initial_robot_positions[robot_idx], target_cargo)
                            
                            # 記錄機器人的路徑和貨物
                            robot_path =  blocking_path
                            
                            # 在到達目的地之前檢查是否需要等待
                            if self.path_passes_through(robot_path, robot_idx,cargo_pos,cargo_dest):
                                print(f"Robot {robot_idx + 1} is waiting for the target position to be free.")
                                continue  # 讓機器人等待，直到下一次迴圈再檢查
                           
                            self.robot_paths.append((f"R{robot_idx + 1}", robot_path))

                            # 更新貨物狀態
                            self.available_cargoes.remove(target_cargo)
                            robot_pos = robot_path[-1]
                            start_time_for_dest = robot_path[-1][2] + 1

                            self.initial_robot_positions[robot_idx] = cargo_dest
                            #self.block_path(path_to_dest)
                            # 鎖定貨物到達的目的地
                            final_pos = robot_path[-1][:2]  # 最後的目的地位置
                            self.cargo_positions[target_cargo]=final_pos
                            self.lock_position(cargo_dest[0], cargo_dest[1], robot_path[-1][2])
            times += 1
            if times >= self.time_steps:
                break

            self.update_cargo_positions()
            
            # 其餘代碼保持不變
            print(f"Available cargos: {self.available_cargoes}")
            print(f"Robot statuses: {self.robot_status}")
            print(f"Cargo positions: {self.cargo_positions}")
        
        end_times = time.time()
        execution_time = end_times - start_times
        print(f"總執行時間：{execution_time:.2f} 秒")
        print(self.robot_paths)

    def path_passes_through(self, path, current_robot_idx,cargo_pos,cargo_dest):
        """檢查路徑是否經過其他機器人的目標位置"""
        # 將可用貨物集合轉換為列表
        available_cargos_list = list(self.available_cargoes)


        # 檢查當前路徑的所有位置
        path_positions = set((x, y) for x, y, t in path)

        # 檢查是否有其他機器人的路徑會經過當前機器人的目標位置
        for robot_name, robot_path in self.robot_paths:
            if robot_name != f"R{current_robot_idx + 1}":  # 忽略當前機器人的路徑
                for (x, y, t) in robot_path:
                    if (x, y) == cargo_dest and t >= self.robot_status[current_robot_idx]-1:  # 確保時間大於當前時間
                        current_time = self.robot_status[current_robot_idx]
                        return True  # 檢測到經過其他機器人的目標位置
         
        

        return False  # 沒有經過


    def update_cargo_positions(self):
        new_positions = list(self.cargo_positions)

        for robot_idx, cargo_idx in self.robot_tasks.items():
            robot_pos = self.initial_robot_positions[robot_idx]

            if cargo_idx in self.cargo_positions:
                if self.cargo_positions[cargo_idx] == robot_pos:
                    new_positions[cargo_idx] = robot_pos

        self.cargo_positions = new_positions
        print(f"Updated cargo positions: {self.cargo_positions}")

    def print_grids(self):
        max_time = max((t for _, path in self.robot_paths for (_, _, t) in path), default=0)
        for t in range(max_time + 1):
            grid = [["." for _ in range(self.grid_size)] for _ in range(self.grid_size)]
            for (robot_idx, path) in self.robot_paths:
                for (x, y, time) in path:
                    if time == t:
                        grid[y][x] = robot_idx
            print(f"Time {t}:")
            for row in grid:
                print(" ".join(row))
            print()

    


In [2]:
import random
import time
import copy
import numpy as np

class GeneticAlgorithm:
    def __init__(self, population_size, num_robots, num_cargos, grid_size, robot_positions, cargo_positions, cargo_destinations, time_steps, generations, mutation_rate=0.1):
        self.population_size = population_size
        self.num_robots = num_robots
        self.num_cargos = num_cargos
        self.grid_size = grid_size
        self.robot_positions = robot_positions
        self.cargo_positions = cargo_positions
        self.cargo_destinations = cargo_destinations
        self.time_steps = time_steps
        self.generations = generations
        self.mutation_rate = mutation_rate

    def initialize_population(self):
        # 隨機初始化每個機器人對每個貨物的權重
        
        population = [np.random.rand(self.num_robots, self.num_cargos) for _ in range(self.population_size)]
        return population

    def allocate_tasks_based_on_weights(self, weights):
        # 初始化可用貨物，過濾掉目標位置為無限大的貨物
        available_cargoes = [
            
            idx for idx in range(len(self.cargo_positions))
            if self.cargo_destinations[idx] != (float('inf'),float('inf'))
        ]  # 只保留有效的貨物索引

        assignments = []  # 儲存每個機器人分配的貨物
        robot_positions_copy = copy.deepcopy(self.robot_positions)
        cargo_positions_copy = copy.deepcopy(self.cargo_positions)
        cargo_destinations_copy = copy.deepcopy(self.cargo_destinations)
        while available_cargoes:  # 當仍有可用的貨物時繼續循環
            for robot_idx in range(len(self.robot_positions)):
                # 根據權重為每個機器人對每個貨物排序
                cargo_preferences = sorted(
                    available_cargoes, 
                    key=lambda c: weights[robot_idx][c], 
                    reverse=True
                )

                for cargo_idx in cargo_preferences:
                    # 檢查貨物是否仍在可用列表中
                    if cargo_idx in available_cargoes:
                        assignments.append((robot_idx, cargo_idx))
                        available_cargoes.remove(cargo_idx)
                        break  # 每個機器人只選擇一個最高優先級的貨物
                if not available_cargoes:
                    break  # 如果所有貨物都已分配，退出循環

        return assignments


    def fitness(self, weights):
        assignments = self.allocate_tasks_based_on_weights(weights)
        #print("Assignments:", assignments)  # Check assignments
        #print("Weights shape:", weights.shape)
        #print("Weights content:", weights)  # Check content

        # Use the allocation results to simulate robot actions
        simulation = GreedyRobotCargoGA(self.grid_size, self.robot_positions, self.cargo_positions, self.cargo_destinations, self.time_steps)
        task_result = simulation.allocate_tasks(assignments)
        if not task_result:
            return float('inf')  # Invalid allocation

        robot_times = {}
        for robot_idx, cargo_idx, path in task_result:
            path_length = len(path)
            robot_times[robot_idx] = robot_times.get(robot_idx, 0) + path_length  # Simplified assignment

        longest_time = max(robot_times.values(), default=0)
        
        return longest_time

    def selection(self, population, fitnesses, num_parents=3):
        selected_parents = []
        for _ in range(len(population)):
            # 錦標賽選擇
            tournament = random.sample(list(zip(population, fitnesses)), num_parents)
            winner = min(tournament, key=lambda x: x[1])[0]  # 根據適應度選擇最優者
            selected_parents.append(winner)
        return selected_parents

    def crossover(self, parents):
        num_parents = len(parents)
        if num_parents < 2:
            raise ValueError("At least two parents are required for crossover.")

        # 隨機選擇一個交叉點
        crossover_point = random.randint(1, self.num_cargos - 1)

        # 初始化子代
        child1 = np.empty_like(parents[0])
        child2 = np.empty_like(parents[0])

        # 進行交叉
        child1[:crossover_point] = parents[0][:crossover_point]
        child2[:crossover_point] = parents[1][:crossover_point]

        child1[crossover_point:] = parents[1][crossover_point:]
        child2[crossover_point:] = parents[0][crossover_point:]
        return child1, child2
    def exchange_mutate(self, individual):
        # 隨機選擇兩個機器人
        robot1_idx, robot2_idx = random.sample(range(self.num_robots), 2)

        # 隨機選擇一個貨物
        cargo_idx = random.randint(0, self.num_cargos - 1)

        # 兌換權重
        individual[robot1_idx, cargo_idx], individual[robot2_idx, cargo_idx] = \
            individual[robot2_idx, cargo_idx], individual[robot1_idx, cargo_idx]

        return individual
    def mutate(self, individual):
        for i in range(self.num_robots):
            for j in range(self.num_cargos):
                if random.random() < self.mutation_rate:
                    individual[i, j] += np.random.normal(0, 0.1)
        if random.random() < self.mutation_rate:  # 這裡用突變率控制兌換突變的機會
            individual = self.exchange_mutate(individual)            
        return individual
    def run(self):
        best = 1000
        start_time = time.time()
        population = self.initialize_population()

        for generation in range(self.generations):
            fitnesses = [self.fitness(weights) for weights in population]
            selected_population = self.selection(population, fitnesses)
            new_population = []

            for i in range(0, len(selected_population), 2):
                if i + 1 < len(selected_population):
                    parents = selected_population[i:i+2]  # 選擇一對父代
                    child1, child2 = self.crossover(parents)  # 獲取兩個子代
                    new_population.extend([self.mutate(child1), self.mutate(child2)])
                else:
                    new_population.append(self.mutate(selected_population[i]))

            population = new_population
            best_fitness = min(fitnesses)
            if best_fitness < best:
                best = best_fitness
            #print(f"Generation {generation}, Best Fitness: {best_fitness}")
        print(f"最短時間：{best} ")
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"總執行時間：{execution_time:.2f} 秒")

        best_individual = population[fitnesses.index(min(fitnesses))]
        return best_individual

grid_size = 5
robot_positions = [(0, 1), (2, 2)]
cargo_positions = [(2, 2), (2, 1), (0, 1)]
cargo_destinations = [(1, 1), (1, 2), (3, 2)]
time_steps = 100
population_size = 50
generations = 20
num_robots = len(robot_positions)
num_cargos = len(cargo_positions)

# 運行
ga = GeneticAlgorithm(population_size, num_robots, num_cargos, grid_size, robot_positions, cargo_positions, cargo_destinations, time_steps, generations, mutation_rate=0.1)
best_allocation_weights = ga.run()


最短時間：13 
總執行時間：5.64 秒
